In [1]:
import sys
sys.path.append('../modules')

from model import Resnet, Bottleneck
from utils import TinyImageNetDataset

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import random
import numpy as np

In [2]:
# Param

batch_size = 128
num_epoch = 200

In [3]:
# CIFAR-10

# download dataset
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root='./data/cifar10', train=True, download = True, transform=transform)

train_mean = torch.stack([data for data, label in trainset]).mean(dim=(0,2,3))
train_std = torch.stack([data for data, label in trainset]).std(dim=(0,2,3))

Files already downloaded and verified


In [5]:
transform_train = transforms.Compose([transforms.RandomCrop(32,padding=4), transforms.RandomHorizontalFlip(), transforms.ToTensor(), transforms.Normalize(train_mean, train_std)])

In [21]:
# label_idx_list = {x : [] for x in range(10)}

# for i in range(len(trainset)):
#     label_idx_list[trainset[i][1]].append(i)
    
# val_set_idx = []
# train_set_idx = []

# for key in label_idx_list.keys():
#     samples = random.sample(label_idx_list[key], 1000)
#     val_set_idx += samples
#     train_set_idx += list(set(label_idx_list[key]) - set(samples))

# print(f'# Train Set : {len(train_set_idx)}\n# Test Set : {len(val_set_idx)}')

# np.save('cifar10_train_idx',train_set_idx)
# np.save('cifar10_val_idx', val_set_idx)

# Train Set : 40000
# Test Set : 10000


In [6]:
trainset = torchvision.datasets.CIFAR10(root='./data/cifar10', train=True, download = False, transform=transform_train)
train_idx = np.load('./cifar10_train_idx.npy')
train_random_sampler = torch.utils.data.SubsetRandomSampler(train_idx)

train_dataloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=False,  sampler=train_random_sampler)
# train_dataloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [7]:
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")

net = Resnet(Bottleneck, [3, 4, 6, 3]).to(device)
net = torch.nn.DataParallel(net, device_ids=[4,6,7]).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)

In [8]:
model_save_path = './model'
loss_history = []

net.train()

DataParallel(
  (module): Resnet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_st

In [9]:
for epoch in range(num_epoch):
    epoch_loss = 0

    for batch_id, (train_x, train_y) in enumerate(tqdm(train_dataloader)):  
        optimizer.zero_grad()
        preds = net(train_x.to(device))

        loss = criterion(preds, train_y.to(device))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    loss_history.append(epoch_loss)

    print(f'epoch : {epoch}, total train loss : {epoch_loss}') 

    with open(f'{model_save_path}/model_epoch_{epoch}.pt','wb') as f:
        torch.save({
            'state' : net.state_dict(),
            'optimizer' : optimizer.state_dict(),
            'epoch' : epoch,
            'loss_history' : loss_history,
        },f)

100%|██████████| 313/313 [01:01<00:00,  5.06it/s]


epoch : 0, total train loss : 599.6908214092255


100%|██████████| 313/313 [00:53<00:00,  5.89it/s]


epoch : 1, total train loss : 485.86606335639954


100%|██████████| 313/313 [00:52<00:00,  5.95it/s]


...

100%|██████████| 313/313 [00:51<00:00,  6.03it/s]


epoch : 179, total train loss : 2.0653303504514042


100%|██████████| 313/313 [00:52<00:00,  5.99it/s]


epoch : 180, total train loss : 2.341559735039482


 44%|████▍     | 138/313 [00:23<00:28,  6.05it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
train_mean

tensor([0.4914, 0.4822, 0.4465])

In [8]:
train_std

tensor([0.2470, 0.2435, 0.2616])